In [4]:
% load_ext autoreload
% autoreload 2
# Plot configurations
% matplotlib inline
from __future__ import print_function
import tensorflow as tf
import numpy as np
from cifar_utils import load_data
from matplotlib import pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Load the raw CIFAR-10 data.
X_train, y_train = load_data(mode='train')

mask = np.arange(X_train.shape[0])
np.random.shuffle(mask)
X_train = X_train[mask]
y_train = y_train[mask]


./data/cifar-10-python.tar.gz already exists. Begin extracting...


In [6]:
################## global contrast normalization ######################
lamda = 10
epsilon = 1e-7

#Because of memory issue, the pre-process of test data has to be after training data
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
contrast = np.sqrt(lamda + np.mean(X_train**2,axis=0))
X_train = X_train / np.maximum(contrast, epsilon)

################## ZCA whitening #########################
temp = []
principal_components = []
for c in range(3):
    X = X_train[:,c*1024:(c+1)*1024]
    cov = np.dot(X.T, X) / (X.shape[0]-1)
    u, s, _ = np.linalg.svd(cov)
    principal_components.append( np.dot(np.dot(u, np.diag(1. / np.sqrt(s + 10e-7))), u.T) )

    # Apply ZCA whitening
    whitex = np.dot(X, principal_components[c])
    temp.append(whitex)

X_train = np.append(temp[0],temp[1],axis=1)
X_train = np.append(X_train,temp[2],axis=1)
X_train = X_train.reshape([50000,3,32,32]).transpose((0,2,3,1))

In [7]:
num_training = 45000
num_validation = 5000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

X_test = []
y_test = []

X_test, y_test = load_data(mode='test')
X_test = X_test.astype(np.float32) - mean_image
contrast = np.sqrt(lamda + np.mean(X_test**2,axis=0))
X_test = X_test / np.maximum(contrast, epsilon)

for c in range(3): 
    X = X_test[:,c*1024:(c+1)*1024]
    whitex = np.dot(X, principal_components[c])
    temp[c] = whitex

X_test = np.append(temp[0],temp[1],axis=1)
X_test = np.append(X_test,temp[2],axis=1)
X_test = X_test.reshape([10000,3,32,32]).transpose((0,2,3,1))

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
#print('Test data shape: ', X_test.shape)
#print('Test labels shape: ', y_test.shape)

Train data shape:  (45000, 32, 32, 3)
Train labels shape:  (45000,)
Validation data shape:  (5000, 32, 32, 3)
Validation labels shape:  (5000,)


In [8]:
del temp
del whitex
del principal_components
del u
del s
del X

In [6]:
# No Regularizer
from binary import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, X_test, y_test,
                     is_binary = False, 
                     is_stochastic = False, 
                     conv_featmap = [128, 128, 256, 256, 512, 512],
                     fc_units = [1024, 1024],
                     conv_kernel_size = [3, 3, 3, 3, 3, 3],
         pooling_size = [2, 2, 2],
         lr_start = 0.001,
         lr_end = 0.0001, 
         epoch = 20,
         batch_size = 50,
         is_drop_out = False,
         verbose = True,
         pre_trained_model = None)

number of batches for training: 900
epoch 1 
average train loss: 0.5482001617219713 ,  average accuracy : 61.815555555555555%
update learning rate:  [0.00089125097]
validation accuracy : 52.26%
* Best accuracy: 52.26%
epoch 2 
average train loss: 0.4592255518502659 ,  average accuracy : 77.76888888888888%
update learning rate:  [0.00079432823]
validation accuracy : 70.2%
* Best accuracy: 70.2%
epoch 3 
average train loss: 0.4355414857798153 ,  average accuracy : 83.11555555555556%
update learning rate:  [0.00070794573]
validation accuracy : 73.94%
* Best accuracy: 73.94%
epoch 4 
average train loss: 0.41752754317389595 ,  average accuracy : 87.72222222222223%
update learning rate:  [0.00063095725]
validation accuracy : 66.0%
epoch 5 
average train loss: 0.4066870464881261 ,  average accuracy : 90.42444444444445%
update learning rate:  [0.00056234124]
validation accuracy : 74.68%
* Best accuracy: 74.68%
epoch 6 
average train loss: 0.39678811921013724 ,  average accuracy : 92.9244444444

In [5]:
# BinaryConnect det.
from binary import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, X_test, y_test,
         is_binary = True, 
         is_stochastic = False, 
         conv_featmap = [128, 128, 256, 256, 512, 512],
         fc_units = [1024, 1024],
         conv_kernel_size = [3, 3, 3, 3, 3, 3],
         pooling_size = [2, 2, 2],
         lr_start = 0.1,
         lr_end = 0.001, 
         epoch = 20,
         batch_size = 50,
         is_drop_out = False,
         verbose = True,
         pre_trained_model = None)

number of batches for training: 900
epoch 1 
average train loss: 0.628632578154405 ,  average accuracy : 48.17777777777778%
update learning rate:  [0.079432823]
validation accuracy : 41.6%
* Best accuracy: 41.6%
epoch 2 
average train loss: 0.5093560012512737 ,  average accuracy : 67.12444444444444%
update learning rate:  [0.063095734]
validation accuracy : 58.52%
* Best accuracy: 58.52%
epoch 3 
average train loss: 0.49222630368338693 ,  average accuracy : 70.66444444444444%
update learning rate:  [0.050118722]
validation accuracy : 65.32%
* Best accuracy: 65.32%
epoch 4 
average train loss: 0.47808745208713743 ,  average accuracy : 73.57111111111111%
update learning rate:  [0.039810713]
validation accuracy : 56.12%
epoch 5 
average train loss: 0.4677721318602562 ,  average accuracy : 75.9%
update learning rate:  [0.031622771]
validation accuracy : 67.25999999999999%
* Best accuracy: 67.25999999999999%
epoch 6 
average train loss: 0.4526451756887966 ,  average accuracy : 79.2911111111

In [9]:
# BinaryConnect stoch.
from binary import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, X_test, y_test,
         is_binary = True, 
         is_stochastic = True, 
         conv_featmap = [128, 128, 256, 256, 512, 512],
         fc_units = [1024, 1024],
         conv_kernel_size = [3, 3, 3, 3, 3, 3],
         pooling_size = [2, 2, 2],
         lr_start = 0.1,
         lr_end = 0.0001, 
         epoch = 50,
         batch_size = 50,
         is_drop_out = False,
         verbose = True,
         pre_trained_model = None)

number of batches for training: 900
epoch 1 
average train loss: 0.7178151541948319 ,  average accuracy : 35.87777777777778%
update learning rate:  [0.087096356]
validation accuracy : 45.46%
* Best accuracy: 45.46%
epoch 2 
average train loss: 0.5397355389263895 ,  average accuracy : 61.85777777777778%
update learning rate:  [0.075857751]
validation accuracy : 55.52%
* Best accuracy: 55.52%
epoch 3 
average train loss: 0.5140823022193379 ,  average accuracy : 66.56888888888889%
update learning rate:  [0.066069335]
validation accuracy : 45.66%
epoch 4 
average train loss: 0.5118471120132341 ,  average accuracy : 67.00444444444445%
update learning rate:  [0.057543986]
validation accuracy : 52.9%
epoch 5 
average train loss: 0.5077765487300026 ,  average accuracy : 68.22444444444444%
update learning rate:  [0.050118715]
validation accuracy : 59.78%
* Best accuracy: 59.78%
epoch 6 
average train loss: 0.5044105234079891 ,  average accuracy : 68.74666666666667%
update learning rate:  [0.043

epoch 50 
average train loss: 0.37963180902931426 ,  average accuracy : 98.24222222222222%
update learning rate:  [9.9999903e-05]
validation accuracy : 85.44%
INFO:tensorflow:Restoring parameters from model/cifar10_1513444905
test accuracy: 84.98%
Traning ends. The best valid accuracy is 85.62%. Model named cifar10_1513444905.
